In [1]:
from recover.datasets.drugcomb_matrix_data import DrugCombMatrix
import torch
import numpy as np
from sklearn.svm import LinearSVR
from scipy.stats import spearmanr
from scipy import stats
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

## Create dataset

In [2]:
dataset = DrugCombMatrix(
    study_name= 'ALMANAC',
    cell_line= 'MCF7',
    fp_bits= 1024,
    fp_radius= 2
)

Dataset loaded.
4463 drug comb experiments among 149 drugs
	 fingeprints with radius 2 and nbits 1024
	 drug features dimension 1173
	 1 cell-lines


In [3]:
# Set seed
seed = 1
torch.manual_seed(seed)
np.random.seed(seed)

# Split
config = {"val_set_prop": 0.2, 
          "test_set_prop": 0.1, 
          "split_valid_train": "pair_level",
          "test_on_unseen_cell_line": False, 
          "target": "bliss_max"}

# The test set does not depend on the seed
train_idxs, val_idxs, test_idxs = dataset.random_split(config)

train_edge_idx = dataset.data.ddi_edge_idx[:, train_idxs]
val_edge_idx = dataset.data.ddi_edge_idx[:, val_idxs]
test_edge_idx = dataset.data.ddi_edge_idx[:, test_idxs]

train_bliss = dataset.data.ddi_edge_bliss_max[train_idxs].numpy()
val_bliss = dataset.data.ddi_edge_bliss_max[val_idxs].numpy()
test_bliss = dataset.data.ddi_edge_bliss_max[test_idxs].numpy()

In [4]:
train_set = torch.cat((dataset.data.x_drugs[train_edge_idx[0]], dataset.data.x_drugs[train_edge_idx[1]]), 
                      dim=1).numpy()
val_set = torch.cat((dataset.data.x_drugs[val_edge_idx[0]], dataset.data.x_drugs[val_edge_idx[1]]), 
                    dim=1).numpy()
test_set = torch.cat((dataset.data.x_drugs[test_edge_idx[0]], dataset.data.x_drugs[test_edge_idx[1]]), 
                    dim=1).numpy()

## SVM hyperparameter search

In [5]:
for tol in [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]:
    for C in [0.0001, 0.001, 0.01, 0.1, 1., 10]:
        comb_svm = LinearSVR(max_iter=1000000, tol=tol, C=C, random_state=0)
        comb_svm.fit(train_set, train_bliss)
        print(tol, C, stats.linregress(comb_svm.predict(val_set), val_bliss).rvalue**2)

0.1 0.0001 0.0810547969200812
0.1 0.001 0.13708287457731294
0.1 0.01 0.17912668476704885
0.1 0.1 0.20540790172961632
0.1 1.0 0.18514747919491603
0.1 10 0.18838635419023417
0.01 0.0001 0.08179467265268225
0.01 0.001 0.1384473143166997
0.01 0.01 0.19202836734006926
0.01 0.1 0.21737626412518152
0.01 1.0 0.21632120693313714
0.01 10 0.21025361637548656
0.001 0.0001 0.08179467265268225
0.001 0.001 0.1384473143166997
0.001 0.01 0.19292154985588775
0.001 0.1 0.21747154858245357
0.001 1.0 0.20956138768107385
0.001 10 0.20886546514478455
0.0001 0.0001 0.08168895299297735
0.0001 0.001 0.13877673227502896
0.0001 0.01 0.1929511202091591
0.0001 0.1 0.21530057385624715
0.0001 1.0 0.21252007542188092
0.0001 10 0.2117044222978486
1e-05 0.0001 0.08173091055892133
1e-05 0.001 0.1389125865975929
1e-05 0.01 0.19286776958358046
1e-05 0.1 0.21580559964427024
1e-05 1.0 0.21288364905940127
1e-05 10 0.21219146853128443


The best parameters are tol=0.001 and C=0.1

## Boosting Tree hyperparameter search

In [9]:
for max_depth in [2, 5, 10, 20]:
    for min_samples_split in [2, 5, 10, 20, 50]:
        for learning_rate in [0.0001, 0.001, 0.01, 0.1, 1.]:
            for max_features in ['auto', 'sqrt', 'log2']:
                reg = GradientBoostingRegressor(n_estimators=100,
                          max_depth=max_depth,
                          min_samples_split=min_samples_split,
                          learning_rate=learning_rate,
                          max_features=max_features,
                          loss='ls')
                reg.fit(train_set, train_bliss)
                print(max_depth, min_samples_split,
                      learning_rate, max_features, stats.linregress(reg.predict(val_set), val_bliss).rvalue**2)

2 2 0.0001 auto 0.1051537211390774
2 2 0.0001 sqrt 0.15871993823752018
2 2 0.0001 log2 0.19408403959361814
2 2 0.001 auto 0.10856470689572481
2 2 0.001 sqrt 0.16884237839411265
2 2 0.001 log2 0.19756955461766934
2 2 0.01 auto 0.23075859949784677
2 2 0.01 sqrt 0.19506814787523974
2 2 0.01 log2 0.1984331266879113
2 2 0.1 auto 0.36571819046776144
2 2 0.1 sqrt 0.3072640950848271
2 2 0.1 log2 0.2561871663451825
2 2 1.0 auto 0.45771055811932065
2 2 1.0 sqrt 0.40594321474554895
2 2 1.0 log2 0.3477115777080695
2 5 0.0001 auto 0.1051537211390774
2 5 0.0001 sqrt 0.15031500188563376
2 5 0.0001 log2 0.19867837307420502
2 5 0.001 auto 0.10856470689572467
2 5 0.001 sqrt 0.1796579455961333
2 5 0.001 log2 0.18649595898542753
2 5 0.01 auto 0.230758599497847
2 5 0.01 sqrt 0.18661448286822557
2 5 0.01 log2 0.20436495099606655
2 5 0.1 auto 0.3656700439252576
2 5 0.1 sqrt 0.30893453396675513
2 5 0.1 log2 0.2694189565904921
2 5 1.0 auto 0.41807347075625184
2 5 1.0 sqrt 0.3917967850841569
2 5 1.0 log2 0.3404

20 2 1.0 log2 0.5688992607773263
20 5 0.0001 auto 0.5540426297586568
20 5 0.0001 sqrt 0.641467432650373
20 5 0.0001 log2 0.5898710825085569
20 5 0.001 auto 0.5591746834528508
20 5 0.001 sqrt 0.639446265142084
20 5 0.001 log2 0.5946785740553325
20 5 0.01 auto 0.624275145391149
20 5 0.01 sqrt 0.6671403815093543
20 5 0.01 log2 0.6262181668969252
20 5 0.1 auto 0.7259064909882899
20 5 0.1 sqrt 0.7266312702622206
20 5 0.1 log2 0.7199871223244872
20 5 1.0 auto 0.6876742803268515
20 5 1.0 sqrt 0.598601291959638
20 5 1.0 log2 0.6241592461120674
20 10 0.0001 auto 0.477796403396145
20 10 0.0001 sqrt 0.6085731478031767
20 10 0.0001 log2 0.5548242663596243
20 10 0.001 auto 0.5261079755556051
20 10 0.001 sqrt 0.6145260487295253
20 10 0.001 log2 0.5572731725893034
20 10 0.01 auto 0.6098159903102823
20 10 0.01 sqrt 0.6419776225694671
20 10 0.01 log2 0.605627199711651
20 10 0.1 auto 0.7189986642819507
20 10 0.1 sqrt 0.7172905387053374
20 10 0.1 log2 0.7106112020116967
20 10 1.0 auto 0.6692197657404576


Best parameters 20 20 0.1 sqrt

## Prediction over 3 seeds for best models

In [6]:
all_r2 = []
all_spear = []

for seed in [2, 3, 4]:
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    train_idxs, val_idxs, test_idxs = dataset.random_split(config)
    train_edge_idx = dataset.data.ddi_edge_idx[:, train_idxs]
    train_bliss = dataset.data.ddi_edge_bliss_max[train_idxs].numpy()
    train_set = torch.cat((dataset.data.x_drugs[train_edge_idx[0]], dataset.data.x_drugs[train_edge_idx[1]]), 
                      dim=1).numpy()

    comb_svm = LinearSVR(max_iter=1000000, tol=0.001, C=0.1, random_state=0)
    comb_svm.fit(train_set, train_bliss)
    
    test_preds = comb_svm.predict(test_set)
    all_r2.append(stats.linregress(test_preds, test_bliss).rvalue**2)
    all_spear.append(spearmanr(test_bliss, test_preds).correlation)
    
print("R2", np.mean(all_r2), np.std(all_r2))
print("Spear", np.mean(all_spear), np.std(all_spear))

R2 0.17143226478348375 0.006290679844989228
Spear 0.4527530524666828 0.004408367459766309


In [7]:
all_r2 = []
all_spear = []

for seed in [2, 3, 4]:
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    train_idxs, val_idxs, test_idxs = dataset.random_split(config)
    train_edge_idx = dataset.data.ddi_edge_idx[:, train_idxs]
    train_bliss = dataset.data.ddi_edge_bliss_max[train_idxs].numpy()
    train_set = torch.cat((dataset.data.x_drugs[train_edge_idx[0]], dataset.data.x_drugs[train_edge_idx[1]]), 
                      dim=1).numpy()

    reg = GradientBoostingRegressor(n_estimators=100,
              max_depth=20,
              min_samples_split=20,
              learning_rate=0.1,
              max_features='sqrt',
              loss='ls')
    
    reg.fit(train_set, train_bliss)
    test_preds = reg.predict(test_set)
    all_r2.append(stats.linregress(test_preds, test_bliss).rvalue**2)
    all_spear.append(spearmanr(test_bliss, test_preds).correlation)
    
print("R2", np.mean(all_r2), np.std(all_r2))
print("Spear", np.mean(all_spear), np.std(all_spear))

R2 0.17240154350005946 0.002411191048605907
Spear 0.45891524869772354 0.0013972159101678335
